In [ ]:
import cv2
from matplotlib import pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from PIL import Image
from PIL import ImageFilter
from matplotlib import animation
from PIL import ImageEnhance
import sys
import os

%matplotlib inline

# for loading array 

from scipy.io import loadmat
import pandas as pd
import numpy as np
import mat4py

import pickle

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:

Libdir = r"C:\Users\nkhefif\Desktop\Scripts\LibrairieWhisk\WhiskCustom"
sys.path.append(Libdir)
os.chdir(Libdir)
from MeasurementsToDF_py3 import MeasurementsToDataframe
data = MeasurementsToDataframe(r"C:\Users\nkhefif\Desktop\\WhiskiTestsVideo\\26_02_test_\\Mouse24_2020-02-18T17.26.39.measurements")

print(data.columns)
display(data)

In [ ]:
#Fonction Resize

def calculateNewSize(width,height,WishForNEwWidth):
    ratio = WishForNEwWidth/width
    Nheight = int(height * ratio)
    Nwidth = int(width * ratio)    
    return Nwidth , Nheight



HandleBEHAV = cv2.VideoCapture(r'C:\Users\nkhefif\Desktop\WhiskiTestsVideo\26_02_test_\Mouse24_2020-02-18T17.26.39.avi')
HandleVSD = cv2.VideoCapture(r'C:\Users\Nicolas\Desktop\Mouse17_Correl_BVH_VSD_test\Session4_trial4\S04-T04_2.mov')

length = int(HandleBEHAV.get(cv2.CAP_PROP_FRAME_COUNT))

_ , IMG0 = HandleBEHAV.read() #Lecture de la première frame de compormement



@interact

def show_image(time=widgets.IntSlider(min=0,max=length-1,step=1,value=1), \
Brightness=widgets.FloatSlider(min=0,max=4,step=0.015,value=1), \
binarization=widgets.FloatSlider(min=-1,max=2,step=0.01,value=-1), \
MinLength =widgets.IntSlider(min=1,max=300,step=1,value=100), \
MinScore = widgets.IntSlider(min=1,max=1000,step=1,value=100), \
Denoise = False, VSD=False, substract= False, EnhanceEdge=False, EnhanceEdgeMore=False ):
    

    HandleBEHAV.set(cv2.CAP_PROP_POS_FRAMES, time)
    _ , BEHi = HandleBEHAV.read()
     
    
    #HandleVSD.set(cv2.CAP_PROP_POS_FRAMES, time)
    #_ , VSDi = HandleVSD.read() 
    #VSDi = cv2.cvtColor(VSDi, cv2.COLOR_BGR2RGB)
   
    
    if Denoise == True:
        
        BEHi = cv2.fastNlMeansDenoisingColored(BEHi) 
        
        #autres manière (marchent pas forcement, je garde au cas ou)
        #gray = cv2.cvtColor(BEHi, cv2.COLOR_RGB2GRAY)
        
        #BEHi = cv2.cvtColor(BEHi, cv2.COLOR_RGB2GRAY)
        #BEHi = cv2.cvtColor(BEHi, cv2.COLOR_RGB2LAB)
       
    
        #BEHi = np.float64(BEHi)
        #noise = np.random.randn(*BEHi[1].shape)*10
        #noisy = [BEHi+noise]
        #noisy = np.uint8(np.clip(BEHi,0,255))
        #dst = cv2.fastNlMeansDenoisingMulti(noisy, 2, 5, None, 4, 7, 35)
       
    if Brightness != 1 :
   
        BEHi = Image.fromarray(BEHi, 'RGB')
        BEHi = ImageEnhance.Brightness(BEHi).enhance(Brightness)
        BEHi = np.array(BEHi)
        
    
    if EnhanceEdge == True: 
        BEHi = Image.fromarray(BEHi, 'RGB')
        BEHi = BEHi.filter(ImageFilter.EDGE_ENHANCE)
        BEHi = np.array(BEHi)
        
    if EnhanceEdgeMore == True: 
        BEHi = Image.fromarray(BEHi, 'RGB')
        BEHi = BEHi.filter(ImageFilter.EDGE_ENHANCE_MORE)
        BEHi = np.array(BEHi)
        
    if substract == True:
        HandleBEHAV.set(cv2.CAP_PROP_POS_FRAMES, 0)
        _ , IMG0 = HandleBEHAV.read()
        BEHi = BEHi-IMG0
   
    if binarization != -1 :
        _ , BEHi = cv2.threshold(BEHi,binarization,255,cv2.THRESH_BINARY)
        
    if VSD == True :
        
        width,height = calculateNewSize(np.shape(VSDi)[1],np.shape(VSDi)[0],np.shape(BEHi)[1])
        VSDi = Image.fromarray(VSDi, 'RGB')
        VSDi = VSDi.resize((width,height), Image.ANTIALIAS)
        output = np.concatenate((VSDi,BEHi),axis=0)
    
    else :
        
        output = BEHi
    
    fig = plt.figure(figsize = (20,30))
    plt.imshow(output, interpolation='none')
    #[test_row.follicle_x, test_row.tip_x ]
    #[test_row.follicle_y, test_row.tip_y ]
    
    i = 0
    
    whiskers_frame = data.loc[(time)]
   # display(whiskers_frame)
    #display(whiskers_frame.shape[0])
    
    for index, row in whiskers_frame.iterrows():
        
    #for i < (whiskers_frame.shape[0]):
        #print(row.length)
        if int(row.length) > MinLength and int(row.score) > MinScore:
            plt.plot([int(row.fol_x), int(row.tip_x)],[int(row.fol_y), int(row.tip_y)], color = 'r' , linewidth = 0.8)
       

            
            
            
 #out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), frame_rate, (frame_width, frame_height))
    
       
    